# Tutorial Julia for Data Science

## 5 Clustering

Questo tutorial riguarda il raggruppare a mostrare dati in gruppi(cluster) tali che un dato sia più simile ai dati del suo cluster che a dati in altri cluster.   
Vedremo 4 modi per fare ciò dal pacchetto Clusters.jl:
+ k-means: partiziona i dati in k gruppi, ognuno dei quali contenente dati più vicini alla media del gruppo (minimizza distanze media-dato)
+ k-medoids: partiziona in k gruppi, minimizzando la distanza di ogni membro di ogni gruppo ad un dato posto come centro del loro gruppo (o moda)  
+ hierarchical clustering: trova una gerarchie di distanze simili
+ DBSCAN: raggruppa in base alla densità dei dati nello spazio


Iniziamo preparando il nostro ambiente e caricando dei dati.

In [30]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

 Activating environment at `C:\Users\i am\Documents\GitHub\stage-Machine-learning\notebook\tutorial\Project.toml`


In [31]:
using Clustering
using VegaLite
using VegaDatasets
using DataFrames
using Statistics
using JSON
using CSV
using Distances

In [32]:
mkpath("downloads")
download("https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv","downloads/newhouses.csv")
download("https://github.com/JuliaAcademy/DataScience/raw/master/data/california-counties.json","downloads/california-counties.json")

houses = CSV.read("downloads/newhouses.csv")
cali_shape = JSON.parsefile("downloads/california-counties.json")
VV = VegaDatasets.VegaJSONDataset(cali_shape,"downloads/california-counties.json")

Vega JSON Dataset

Il dataset che abbviamo caricato riguarda i costi delle abitazioni in california abbinati alla loro locazione (latitudine, longitudine) mentre il JSON è una rappresetazione delle county della californioa che ci serviurà a rappresentare graficamente i dati spazialmente.  
Possiamo vedere nello spazio i dati dando un impressione di dove sono le abitazioni più costose con una heatmap:

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="median_house_value:q"
                    
)

Possiamo anche diminuire la precisione raggruppando i valori delle case in classi con incrementi di 50k

In [ ]:
bucketprice = Int.(div.(houses[!,:median_house_value],50000))
insertcols!(houses,3,:cprices=>bucketprice)

@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="cprices:n"
                    
)

### K-means clustering

K-means partiziona il dataset (preso come matrice) in k gruppi, ognuno di questi gruppi conterrà i dati più vicini alla media del gruppo stesso.  
Proviamo a lanciare kmeans (dal pkg clustering) raggruppando in 10 gruppi basati su latitudine/longitudine.

Notare come abbiam dovuto convertire il df in matrice per kmeans.

In [ ]:
X = houses[!, [:latitude,:longitude]]
C = kmeans(Matrix(X)', 10) # kemans prende matrici come input quindi dobbiamo convertire dataframe X into matrice
insertcols!(houses,3,:cluster10=>C.assignments)#aggiunge colonne clkuster10
X

Possiamo vedere una rappresentazione grafica di questi gruppi:

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="cluster10:n"#colore in base al cluster
                    
)

### K-medoids clustering

K-medoids genera k gruppi contenenti i dati più vicini a un membro del gruppo posto come centro del gruppo stesso.  
Per usare la funzione kmedoids di Clustering.jl dobbiamo ottenere una matrice delle distanze per i dati.  
Generiamola a partire dalle distanze euclidiane dei dati:

In [ ]:
xmatrix = Matrix(X)'
D = pairwise(Euclidean(), xmatrix, xmatrix,dims=2) # facciamla secondo distanza euclidiana

K = kmedoids(D,10)# 10 gruppi
insertcols!(houses,3,:medoids_clusters=>K.assignments)#aggiungiamo altra colonna

Una volta ottenuto ed aggiunto all df i dati, possiamo proiettarli sulla mappa della california, come prima:

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="medoids_clusters:n"
                    
)

### Clustering gerarchico
Un algoritmo di clustering gerarchico restituisce una gerarchia di vicinanze dei dati.  
Questo può avvenire seconbdo due strategie:
+ strategia agglomerativa: 
 1. all inizio ogni dato è un cluster
 2. i cluster più vicini vengono agglomerati
 3. ripeti step2 finchè tutti i dati sono sullo stesso gruppo
 4. il risultato è il dendogramma che spiega l'ordine in cui sono stati aggruppati i dati
+ strategia divisiva:  simile all'agglomerativa ma partendo da un solo gruppo conetnente tutti i dati a cui vengono rimossi i dati in base a quali sono più distanti  

In [ ]:
K = hclust(D)
L = cutree(K;k=10)
insertcols!(houses,3,:hclust_clusters=>L)

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="hclust_clusters:n"
                    
)

Vediamo che non appaiono gerarchie particolari, i dati sono a grandi linee equidistanti.

### DBSCAN
Density-based spatial clustering of applications with noise (DBSCAN) è una tecnica di clustering che, dati dei dati spaziali, restituisce aggruppamenti basati su quali gruppi di dati sono più aggruppati vicini (maggior numero di vicini), marcando inoltre outlier, soli in aree a bassa densità, i cui vicini più vicini sono particolarmente lontani.  
Vediamo anche questo dal package Clustering.jl.  
Anche per questo ci serve una matrice di distanze, questa volta usiamo la distanza quadratica per diminuire la varianza.

In [ ]:
using Distances
dclara = pairwise(SqEuclidean(), Matrix(X)',dims=2)#mat distanze fatta con dist euclid quadrata questa volta
D = pairwise(Euclidean(), xmatrix, xmatrix,dims=2)
L = dbscan(dclara, 0.05, 10)
@show length(unique(L.assignments))

insertcols!(houses,3,:dbscanclusters=>L.assignments)

L.assignments è il gruppo assegnato, unique(l.assignments) ci ritorna quindi quanti gruppi sono stati generati.  
Vediamo ora in un grafico.

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
    
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="dbscanclusters:n"
                    
)

In [ ]:
housesdef = CSV.read("downloads/newhouses.csv")
names(housesdef)